#  train emotion (fer2013)

###  load data

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("./datasets/fer2013/fer2013.csv")

In [3]:
data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


### training

In [4]:
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from model.cnn import mini_XCEPTION

In [5]:
from utils.dataset import DataManager
from utils.dataset import split_data
from utils.preprocessor import preprocess_input

In [6]:
# parameters
batch_size = 32
num_epochs = 100
input_shape = (64, 64, 1)
validation_split = .2
verbose = 1
num_classes = 7
patience = 50
base_path = './datasets/'

In [7]:
# data generator
data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)

In [8]:
# model parameters/compilation
model = mini_XCEPTION(input_shape, num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['acc']) # version error ('accuary' >> 'acc')
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 62, 62, 8)    72          input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 62, 62, 8)    32          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 62, 62, 8)    0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [9]:
datasets = ['fer2013']
for dataset_name in datasets:
    print('Training dataset:', dataset_name)

    # callbacks
    log_file_path = base_path + dataset_name + '_emotion_training.log'
    csv_logger = CSVLogger(log_file_path, append=False)
    early_stop = EarlyStopping('val_loss', patience=patience)
    reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1,
                                  patience=int(patience/4), verbose=1)
    trained_models_path = base_path + dataset_name + '_mini_XCEPTION'
    model_names = trained_models_path + '.{epoch:02d}-{val_acc:.2f}.hdf5'
    model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose=1,
                                                    save_best_only=True)
    callbacks = [model_checkpoint, csv_logger, early_stop, reduce_lr]

    # loading dataset
    data_loader = DataManager(dataset_name, image_size=input_shape[:2])
    faces, emotions = data_loader.get_data()
    faces = preprocess_input(faces)
    num_samples, num_classes = emotions.shape
    train_data, val_data = split_data(faces, emotions, validation_split)
    train_faces, train_emotions = train_data
    model.fit_generator(data_generator.flow(train_faces, train_emotions,
                                            batch_size),
                        steps_per_epoch=len(train_faces) / batch_size,
                        epochs=num_epochs, verbose=1, callbacks=callbacks,
                        validation_data=val_data)

Training dataset: fer2013
Epoch 1/100
897/897 [============================>.] - ETA: 0s - loss: 1.8148 - acc: 0.3024
Epoch 00001: val_loss improved from inf to 1.63840, saving model to ./datasets/fer2013_mini_XCEPTION.01-0.39.hdf5
898/897 [==============================] - 538s 599ms/step - loss: 1.8147 - acc: 0.3024 - val_loss: 1.6384 - val_acc: 0.3948
Epoch 2/100
897/897 [============================>.] - ETA: 0s - loss: 1.5310 - acc: 0.4245
Epoch 00002: val_loss improved from 1.63840 to 1.45486, saving model to ./datasets/fer2013_mini_XCEPTION.02-0.43.hdf5
898/897 [==============================] - 502s 559ms/step - loss: 1.5312 - acc: 0.4245 - val_loss: 1.4549 - val_acc: 0.4310
Epoch 3/100
897/897 [============================>.] - ETA: 0s - loss: 1.4023 - acc: 0.4723
Epoch 00003: val_loss did not improve from 1.45486
898/897 [==============================] - 426s 474ms/step - loss: 1.4020 - acc: 0.4725 - val_loss: 1.4979 - val_acc: 0.4550
Epoch 4/100
897/897 [===================

Epoch 29/100
897/897 [============================>.] - ETA: 0s - loss: 0.9829 - acc: 0.6363
Epoch 00029: val_loss did not improve from 1.05931
898/897 [==============================] - 515s 574ms/step - loss: 0.9828 - acc: 0.6363 - val_loss: 1.0965 - val_acc: 0.6124
Epoch 30/100
897/897 [============================>.] - ETA: 0s - loss: 0.9782 - acc: 0.6362
Epoch 00030: val_loss did not improve from 1.05931
898/897 [==============================] - 478s 533ms/step - loss: 0.9783 - acc: 0.6362 - val_loss: 1.0932 - val_acc: 0.6016
Epoch 31/100
897/897 [============================>.] - ETA: 0s - loss: 0.9746 - acc: 0.6377
Epoch 00031: val_loss improved from 1.05931 to 1.05470, saving model to ./datasets/fer2013_mini_XCEPTION.31-0.62.hdf5
898/897 [==============================] - 512s 570ms/step - loss: 0.9747 - acc: 0.6376 - val_loss: 1.0547 - val_acc: 0.6154
Epoch 32/100
897/897 [============================>.] - ETA: 0s - loss: 0.9719 - acc: 0.6388
Epoch 00032: val_loss improved fr

897/897 [============================>.] - ETA: 0s - loss: 0.8976 - acc: 0.6700
Epoch 00058: val_loss did not improve from 1.01540
898/897 [==============================] - 461s 513ms/step - loss: 0.8978 - acc: 0.6699 - val_loss: 1.1069 - val_acc: 0.6069
Epoch 59/100
897/897 [============================>.] - ETA: 0s - loss: 0.8953 - acc: 0.6680
Epoch 00059: val_loss did not improve from 1.01540
898/897 [==============================] - 458s 510ms/step - loss: 0.8950 - acc: 0.6682 - val_loss: 1.0241 - val_acc: 0.6269
Epoch 60/100
897/897 [============================>.] - ETA: 0s - loss: 0.8957 - acc: 0.6700
Epoch 00060: val_loss did not improve from 1.01540

Epoch 00060: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
898/897 [==============================] - 458s 510ms/step - loss: 0.8961 - acc: 0.6698 - val_loss: 1.0388 - val_acc: 0.6305
Epoch 61/100
897/897 [============================>.] - ETA: 0s - loss: 0.8448 - acc: 0.6871
Epoch 00061: val_loss improved 

897/897 [============================>.] - ETA: 0s - loss: 0.7817 - acc: 0.7122
Epoch 00086: val_loss did not improve from 0.96986
898/897 [==============================] - 421s 469ms/step - loss: 0.7814 - acc: 0.7123 - val_loss: 0.9728 - val_acc: 0.6506
Epoch 87/100
897/897 [============================>.] - ETA: 0s - loss: 0.7852 - acc: 0.7111
Epoch 00087: val_loss did not improve from 0.96986
898/897 [==============================] - 405s 452ms/step - loss: 0.7850 - acc: 0.7112 - val_loss: 0.9723 - val_acc: 0.6510
Epoch 88/100
897/897 [============================>.] - ETA: 0s - loss: 0.7851 - acc: 0.7085
Epoch 00088: val_loss did not improve from 0.96986
898/897 [==============================] - 1157s 1s/step - loss: 0.7851 - acc: 0.7085 - val_loss: 0.9735 - val_acc: 0.6498
Epoch 89/100
897/897 [============================>.] - ETA: 0s - loss: 0.7832 - acc: 0.7141
Epoch 00089: val_loss did not improve from 0.96986
898/897 [==============================] - 440s 489ms/step - los